# Multi-objective Fitting and Parameter Importance

In [1]:
# IMPORTING LIBRARIES
import os
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import io as sio
import glob
import warnings; warnings.filterwarnings('ignore')
import distance_FC
from distance_FC import distance_FC
from sklearn.metrics import mean_squared_error

# IMPORT THE EMPIRICAL DATA FROM THE ASD SUBJECT (RESTING STATE EEG) and the simulation data
#rest = np.load(r"C:\Users\leona\Leo_Python\rest_asd.npy")
rest = np.load("/home/l_martinelli/Michelangelo/FITTING/rest_asd.npy")
n_channels = rest.shape[0]
rest.shape
#file_names = glob.glob("/home/l_martinelli/Michelangelo/subResults/mat_eeg/*.mat")  #extract a list of file names in the directory
file_names = glob.glob("/home/l_martinelli/Michelangelo/Parameter_fitting_2024-08-28/Results/RUN03-BestFC_Point_Linear_Scale_PSD_Optimization_vaying_AaBb/*.mat")  #extract a list of file names in the directory
n_trials = len(file_names)   # number of simulation trials 
a = str(file_names[0])  #load one data to extract the information about n_points
sim_surface = sio.loadmat(a)
simm = np.array(sim_surface['eeg'])
n_points = int(simm.shape[0])  #assuming the shapes of simulations is [n_points, n_channels]
n_ch = 64  
simulation = np.zeros([n_points, n_ch, n_trials])  # array to be filled with simulation in the shape [number of points, number of channels, number of trials]
sim = np.zeros([n_channels, n_points, n_trials])  # array with the mask [number of channels (in common with empirical data), number of points, number of trials]
for i in range (n_trials):
    a = str(file_names[i])
    sim_surface = sio.loadmat(a)
    simulation[:,:,i] = np.array(sim_surface['eeg'])
    sim_m = simulation[:,:,i].mean(axis=1)
    for j in range (n_ch):
        simulation[:,j,i] = simulation[:,j,i] - sim_m
        
# MASK FOR THE EXTRACTION OF THE 16 CHANNELS IN COMMON WITH THE EMPIRICAL DATA
#mask_old = (4,12,20,26,14,22,32,37,47,30,33,39,49,57,63,51)
mask = (42,21,2,9,36,33,10,16,0,3,45,49,28,61,8,35)
for i in range (n_channels):
    sim[i,:,:] = simulation[:,mask[i],:]

# DELETE FIRST 500 POINTS

sim = sim[:,500:,:]
sim.shape

#compute and return the pearson correlation matrix for the empirical data (in shape [n_channels,n_channels]) or simulated [n_channels,n_channels,n_trials] 
#and the array of the triangular part of the matrix 
# data are supposed to be in the shape [n_channels, n_points, n_trials]

def pearson_connectivity(data):
    data_shape = data.shape
    n_channels = data_shape[0]
    n_array = int((n_channels)*(n_channels - 1)/2)
    n_trials = data_shape[-1]
    n_points = data_shape[1]
    #distinguish the case between one trial and many trials (simulations)
    if n_trials == n_points:
        functional_matrix = np.zeros([n_channels,n_channels])
        functional_array = np.zeros([n_array])
        pearson_correlation = pd.DataFrame(data=data[:, :]).T.corr()
        functional_matrix = pearson_correlation.to_numpy()
        #extract the triangular part
        z = -1
        for i in range (1,n_channels):
            for j in range (0,i):
                z = z + 1 
                functional_array[z] = functional_matrix[i,j]
    else:
        functional_matrix = np.zeros([n_channels,n_channels,n_trials])
        functional_array = np.zeros([n_array,n_trials])
        for i in range (n_trials):
            pearson_correlation = pd.DataFrame(data=data[:, :, i]).T.corr()
            functional_matrix[:,:,i] = pearson_correlation.to_numpy()
            z = -1
            for j in range (1,n_channels):
                for k in range (0,j):
                    z = z + 1
                    functional_array[z,i] = functional_matrix[j,k,i]
    return functional_matrix, functional_array
        


#compute and return the correlation between the functional connectivity matrices: simulated vs recorded for each simulation trial

def fit_connectivity(simulation):
    simulated_connectivity = pearson_connectivity(simulation)[1]
    recorded_connectivity = pearson_connectivity(rest)[1]
    simulation_shape = simulation.shape
    n_channels = simulation_shape[0]
    n_array = int((n_channels)*(n_channels - 1)/2)
    n_points = simulation_shape[1]
    n_trials = simulation_shape[-1]
    D = np.zeros([n_array,2])  # contains two vectors: the simulated and the empirical data to be compared
    if n_trials == n_points:
        correlation_fit = np.zeros([1])
        B = np.asarray(simulated_connectivity[:]).reshape(-1)
        C = np.asarray(recorded_connectivity[:]).reshape(-1)
        D[:,0] = B
        D[:,1] = C
        diff_corr_ = pd.DataFrame(data=D[:, :]).corr()
        diff_corr_ = diff_corr_.to_numpy()
        correlation_fit = diff_corr_[0,1]
    else:
        correlation_fit = np.zeros([n_trials])
        for i in range (n_trials):
            B = np.asarray(simulated_connectivity[:,i]).reshape(-1)
            C = np.asarray(recorded_connectivity[:]).reshape(-1)
            D[:,0] = B
            D[:,1] = C
            diff_corr_ = pd.DataFrame(data=D[:, :]).corr()
            diff_corr_ = diff_corr_.to_numpy()
            correlation_fit[i] = diff_corr_[0,1]
    return correlation_fit

#fit for the uploaded simulations 'sim'
fit0 = fit_connectivity(sim)
fit0_ = fit0[~np.isnan(fit0)]
max0 = fit0_.max()
best0 = np.where(fit0 == max0)
best0 = np.array(best0)


#COMPUTE DISTANCE BETWEEN FC MATRICES WITH THE GEODESIC DISTANCE
def fit_geodesic(simulation):
    FC1 = pearson_connectivity(rest)[0]
    d_geodesic = np.zeros([n_trials])
    for i in range(n_trials):
        if fit0[i] > 0:    #discard null simulations
            FC2 = pearson_connectivity(simulation[:,:,i])[0]
            dist = distance_FC(FC1, FC2)
            d_geodesic[i] = dist.geodesic()
    return d_geodesic



In [13]:
# compute the PSD of the data and the extract periodic and aperiodic compents with FOOOF algorithm
#compute PSD with Welch's method via median average, the signal must be in the form [n_channels,n_points]

def PSD(data):
    
    n_channels = data.shape[0]
    fs = 256
    f, spec = signal.welch(data[0,:],fs = fs,nperseg = 1024)
    f_nyq = spec.shape[0]
    spectrum = np.zeros([f_nyq ,n_channels])
    for i in range (n_channels):
        f, spectrum[:,i] = signal.welch(data[i,:],fs = fs, average = 'median',nperseg = 1024)

    spectrum = spectrum[:160,:]
    f = f[:160]
    spectrum_mean = spectrum.mean(axis = 1)
    spectrum_max = spectrum_mean.max()
    spectrum_norm = spectrum_mean/spectrum_max

    return spectrum_norm


def fit_psd(simulation):
    n_trials = simulation.shape[2]
    sim_psd = np.zeros([160,n_trials])
    mse_psd = np.zeros([n_trials]) 
    rec_psd = PSD(rest)  #compute psd and aperiodic exponent of empirical data
    for j in range(n_trials):
        sim_psd[:,j] = PSD(simulation[:,:,j]) #compute psd and aperiodic exponent of simulated data
        if sim_psd[0,j] > 0 :
            mse_psd[j] = mean_squared_error(sim_psd[:,j], rec_psd)
    return mse_psd, sim_psd



In [3]:
#compute the multi-objective fit and the corresponding simulation labels

fit0 = fit_connectivity(sim)
fit0_ = fit0[~np.isnan(fit0)]
max0 = fit0_.max()
best0 = np.where(fit0 == max0)
best0 = np.array(best0)

fit1, psd1 = fit_psd(sim)
fit1_ = fit1[~np.isnan(fit1)]
max1 = fit1_.max()
best1 = np.where(fit1 == max1)
best1 = np.array(best1)

In [ ]:
def PSD_COMPONENTS(psd):
    fs = 256
    f, spec = signal.welch(rest[0,:],fs = fs, nperseg = 512)
    f = f[:80]
    alpha = np.zeros([5075])
    peak = np.zeros([5075])
    n_p = np.zeros([5075])
    for i in range (5075):
         if psd[0,i] > 0:
            # Initialize FOOOF object
            fm = FOOOF()#aperiodic_mode='knee')
            # Define frequency range across which to model the spectrum
            freq_range = [1, 40]
            # Model the power spectrum with FOOOF, and print out a report
            #fm.report(f, spectrum_norm, freq_range)
            fm.fit(f, psd[:,i], freq_range)
            #extract aperiodic component
            alpha[i] = fm.aperiodic_params_[1]
            #and periodic component (first peak in the PSD)
            p = fm.peak_params_
            n_p[i] = fm.n_peaks_
            if n_p[i] > 0:
                peak[i] = p[0,0]
    return peak, alpha

In [ ]:
#PARAMETER IMPORTANCE
#ONCE A SIMULATION GRID SEARCH IS RUNNED FITTING VALUES TOGETHER WITH PARAMETERS ARRAYS 
#ARE USED FOR PARAMETR IMPORTANCE ON BOTH THE FITTING RESULTS AND ON FEATURES OF THE WHOLE BRAIN ACTIVITY

#parameters argument. these depend on the particular grid search implemented
arg = ['jrm.a',
 'jrm.b',
 'white_matter_speed',
 'local_connectivity_sigma',
 'local_coupling_strength',
 'jrm.mu',
 'white_matter_coupling.a']

#train a random forest on the data and compute parameter gini importance

X = pd.DataFrame(par.T, columns = arg)     #par is the array of parameters  with parameters arguments in 'arg' list, result of the grid search simulation
y1 = alpha
y2 = peak
y3 = fit0
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.25)  #use y1 for periodic peak, y2 for aperiodic exponent or y3 for FC fit
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)
X_train_random = X_train.copy()
X_train_random["RANDOM"] = np.random.RandomState(42).randn(X_train.shape[0])
rf_random = RandomForestRegressor(n_estimators=100, random_state=42)
rf_random.fit(X_train_random, y_train)
global_importances_random = pd.Series(rf_random.feature_importances_, index=X_train_random.columns)

#validate computing permutation importance
result = permutation_importance(
    rf, X_train, y_train, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)

#extend analysis considering shapley values
xplainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")
shap.summary_plot(shap_values, X_test, title = "Shapley value feature importances on periodic peak")